In [2]:
import gspread
import pandas as pd
import requests
from oauth2client.service_account import ServiceAccountCredentials
from bs4 import BeautifulSoup
from unidecode import unidecode
import time

In [4]:
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name('C:/Users/Jedrek/Downloads/serene-cathode-397013-f6b9fc969ce8.json', scope)
client = gspread.authorize(credentials)
# Load Google Sheets credentials
spreadsheet = client.open_by_key('1iqsF5kwcy7hxWpuJXyMniZOrB6iw3hSz56QkgIfWazQ')

# Get a list of all sheet names
sheet_list = spreadsheet.worksheets()

# Initialize a dictionary to store DataFrames
dataframes = {}

# Loop through each sheet and convert to DataFrame
for sheet in sheet_list:
    if sheet.title != 'Plan' and sheet.title != 'Glossary' and sheet.title != 'Mapping' and sheet.title != 'Arkusz30' :
        data = sheet.get_all_values()
        df = pd.DataFrame(data[1:], columns=data[0])  # Assuming the first row contains column headers
        dataframes[sheet.title] = df

# Now you have a dictionary of DataFrames, where keys are sheet names and values are DataFrames
# You can access individual DataFrames using dataframes['Sheet Name']

# For example, to print the first few rows of a sheet:


In [6]:
all_players = []

print(dataframes.keys())
for df_key, df in dataframes.items():
    all_players += df['Player'].tolist()

all_players


dict_keys(['players_general_stats', 'goalkeeping_stats', 'defensive_stats', 'shot_creating_stats', 'shooting_stats'])


['Brenden Aaronson',
 'George Abbott',
 'Che Adams',
 'Tyler Adams',
 'Tosin Adarabioyo',
 'Nayef Aguerd',
 'Naouirou Ahamada',
 'Rayan Aït Nouri',
 'Kristoffer Ajer',
 'Manuel Akanji',
 'Nathan Aké',
 'Marc Albrighton',
 'Thiago Alcántara',
 'Carlos Alcaraz',
 'Trent Alexander-Arnold',
 'Alisson',
 'Dele Alli',
 'Miguel Almirón',
 'Julián Álvarez',
 'Daniel Amartey',
 'Samuel Amo-Ameyaw',
 'Joachim Andersen',
 'Elliot Anderson',
 'Jaidon Anthony',
 'Michail Antonio',
 'Antony',
 'Cameron Archer',
 'Alphonse Areola',
 'Joe Aribo',
 'Adam Armstrong',
 'Stuart Armstrong',
 'Kepa Arrizabalaga',
 'Pierre-Emerick Aubameyang',
 'Ludwig Augustinsson',
 'Serge Aurier',
 'Taiwo Awoniyi',
 'Yasin Ayari',
 'André Ayew',
 'Jordan Ayew',
 'Luke Ayling',
 'César Azpilicueta',
 'Benoît Badiashile',
 'Leon Bailey',
 'Stefan Bajcetic',
 'Dominic Ballard',
 'Patrick Bamford',
 'Shandon Baptiste',
 'Harvey Barnes',
 'Gavin Bazunu',
 'Mads Bech Sørensen',
 'Jan Bednarek',
 'Jan Bednarek',
 'Donny van de B

In [7]:
unique_list = list(dict.fromkeys(all_players))
len(unique_list)


1686

In [57]:
transfermarkt = {'player':[], 'club':[],'value':[], 'age':[]}
i = 0
for player in unique_list:
        cleaned_player = player.replace(' ','+')
        print(player)
        url = 'https://www.transfermarkt.com/schnellsuche/ergebnis/schnellsuche?query='+cleaned_player
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'
                }
        # Send a GET request to the URL with the headers
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Check if the request was successful
        if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
                transfermarkt['player'].append(player)
                td_tags = soup.find_all('td')
                if len(td_tags) > 0:
                        age = td_tags[6].text
                        transfermarkt['age'].append(age)
                        print(td_tags[3])
                        a_tags = td_tags[0].find_all(lambda tag: tag.name == 'a' and 'verein' in tag.get('href', '').lower())
                        print(a_tags)
                        if len(a_tags) > 0:
                                club = a_tags[0].text
                        elif td_tags[3].text == 'Retired':
                                club = 'Retired'
                        else:
                                club = 'player_not_found'

                
                else:
                        club = 'player_not_found'
                        transfermarkt['age'].append('player_not_found')
                transfermarkt['club'].append(club)
                value = soup.find_all('td',class_= 'rechts hauptlink')
                if len(value) > 0:
                        transfermarkt['value'].append(value[0].text)
                else:
                        transfermarkt['value'].append('player_not_found')
                i = i+1
                print(club)
                print('Completion - ', (i/1686)*100, "%")


Brenden Aaronson
<td><a href="/1-fc-union-berlin/startseite/verein/89" title="1.FC Union Berlin">1.FC Union Berlin</a></td>
[<a href="/1-fc-union-berlin/startseite/verein/89" title="1.FC Union Berlin">1.FC Union Berlin</a>]
1.FC Union Berlin
Completion -  0.05931198102016608 %
George Abbott
<td><a href="/tottenham-hotspur-u18/startseite/verein/6928" title="Tottenham Hotspur U18">Tottenham Hotspur U18</a></td>
[<a href="/tottenham-hotspur-u18/startseite/verein/6928" title="Tottenham Hotspur U18">Tottenham Hotspur U18</a>]
Tottenham Hotspur U18
Completion -  0.11862396204033215 %
Che Adams
<td><a href="/fc-southampton/startseite/verein/180" title="Southampton FC">Southampton FC</a></td>
[<a href="/fc-southampton/startseite/verein/180" title="Southampton FC">Southampton FC</a>]
Southampton FC
Completion -  0.1779359430604982 %
Tyler Adams
<td><a href="/afc-bournemouth/startseite/verein/989" title="AFC Bournemouth">AFC Bournemouth</a></td>
[<a href="/afc-bournemouth/startseite/verein/989" 

In [60]:
i = 0
df = None
df = pd.DataFrame(transfermarkt)
for key, item in transfermarkt.items():
    print(len(item))
    print(key)

print(transfermarkt)
df
def new_func(df):
    pd.set_option('display.max_columns', None)
    pd.set_option('display.max_rows', None)
    print(df)

new_func(df) 
print(transfermarkt['player'])
for element in unique_list:
    if element not in transfermarkt['player']:
        i = i+1
        print(element)
print(i)



1686
player
1686
club
1686
value
1686
age
{'player': ['Brenden Aaronson', 'George Abbott', 'Che Adams', 'Tyler Adams', 'Tosin Adarabioyo', 'Nayef Aguerd', 'Naouirou Ahamada', 'Rayan Aït Nouri', 'Kristoffer Ajer', 'Manuel Akanji', 'Nathan Aké', 'Marc Albrighton', 'Thiago Alcántara', 'Carlos Alcaraz', 'Trent Alexander-Arnold', 'Alisson', 'Dele Alli', 'Miguel Almirón', 'Julián Álvarez', 'Daniel Amartey', 'Samuel Amo-Ameyaw', 'Joachim Andersen', 'Elliot Anderson', 'Jaidon Anthony', 'Michail Antonio', 'Antony', 'Cameron Archer', 'Alphonse Areola', 'Joe Aribo', 'Adam Armstrong', 'Stuart Armstrong', 'Kepa Arrizabalaga', 'Pierre-Emerick Aubameyang', 'Ludwig Augustinsson', 'Serge Aurier', 'Taiwo Awoniyi', 'Yasin Ayari', 'André Ayew', 'Jordan Ayew', 'Luke Ayling', 'César Azpilicueta', 'Benoît Badiashile', 'Leon Bailey', 'Stefan Bajcetic', 'Dominic Ballard', 'Patrick Bamford', 'Shandon Baptiste', 'Harvey Barnes', 'Gavin Bazunu', 'Mads Bech Sørensen', 'Jan Bednarek', 'Donny van de Beek', 'Asmir Be

In [66]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print(df) 

df.to_csv('transfermarkt-values.csv')

                              player                              club  \
0                   Brenden Aaronson                 1.FC Union Berlin   
1                      George Abbott             Tottenham Hotspur U18   
2                          Che Adams                    Southampton FC   
3                        Tyler Adams                   AFC Bournemouth   
4                   Tosin Adarabioyo                         Fulham FC   
5                       Nayef Aguerd                   West Ham United   
6                   Naouirou Ahamada                    Crystal Palace   
7                    Rayan Aït Nouri                  player_not_found   
8                    Kristoffer Ajer                      Brentford FC   
9                      Manuel Akanji                   Manchester City   
10                        Nathan Aké                   Manchester City   
11                   Marc Albrighton                    Leicester City   
12                  Thiago Alcántara  

In [64]:
df_not_found = df.loc[(df['club'] == 'player_not_found') ]
df_retired = df.loc[(df['club'] == 'Retired')]
print(df_not_found)
names = []


print(len(df_not_found))


for name in df_not_found['player']:
    string = name.split()
    names.append(string[-1])


print(names)
df_not_found['surname'] = names
print(df_not_found)
print(df_retired)

                         player              club             value  \
7               Rayan Aït Nouri  player_not_found  player_not_found   
57    Victor Bernth Kristiansen  player_not_found  player_not_found   
194              Degnand Gnonto  player_not_found  player_not_found   
217              Hwang Hee-chan  player_not_found  player_not_found   
222               Son Heung-min  player_not_found  player_not_found   
230             Pierre Højbjerg  player_not_found  player_not_found   
355             Mykhailo Mudryk  player_not_found  player_not_found   
564               Emanuel Aiwum  player_not_found  player_not_found   
565      Jean-Daniel Akpa-Akpro  player_not_found  player_not_found   
690            Mickaël Cuisance  player_not_found  player_not_found   
761       Pablo Galdames Millán  player_not_found  player_not_found   
779           Christian Gytkjær  player_not_found  player_not_found   
799           Antonio Iervolino  player_not_found  player_not_found   
812   

C:\Users\Jedrek\AppData\Local\Temp\ipykernel_36728\942049013.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_found['surname'] = names


In [49]:
transfermarkt1 = {'player':[], 'club':[],'value':[], 'age':[]}
player = 'Cafu'
cleaned_player = player.replace(' ','+')
print(player)
url = 'https://www.transfermarkt.com/schnellsuche/ergebnis/schnellsuche?query='+cleaned_player
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36'
                }
        # Send a GET request to the URL with the headers
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')


        # Check if the request was successful
if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
                transfermarkt1['player'].append(player)
                td_tags = soup.find_all('td')
                if len(td_tags) > 0:
                        age = td_tags[6].text
                        transfermarkt1['age'].append(age)
                        print(td_tags[3])
                        a_tags = td_tags[0].find_all(lambda tag: tag.name == 'a' and 'verein' in tag.get('href', '').lower())
                        print(a_tags)
                        if len(a_tags) > 0:
                                club = a_tags[0].text
                        elif td_tags[3].text == 'Retired':
                                club = 'Retired'
                        else:
                                club = 'player_not_found'
                                transfermarkt1['age'].append('player_not_found')
                
                else:
                        transfermarkt1['age'].append('player_not_found')
                        club = 'player_not_found'

                transfermarkt1['club'].append(club)
                value = soup.find_all('td',class_= 'rechts hauptlink')
                if len(value) > 0:
                        transfermarkt1['value'].append(value[0].text)
                else:
                        transfermarkt1['value'].append('player_not_found')

                print(club)


Cafu
<td>Retired</td>
[]
Retired
